In [ ]:
import keras
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization, Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

In [46]:
train = pd.read_csv(r'input/train.csv')
test = pd.read_csv(r'input/test.csv')

In [48]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
y = train["label"]
train.drop(["label"], inplace=True, axis=1) #删去label列
X = train

In [50]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.9, random_state=42)

In [51]:
X_train = X_train.values.reshape(-1, 28, 28, 1).astype("float32") / 255
X_val = X_val.values.reshape(-1, 28, 28, 1).astype("float32") / 255
X_test= test.values.reshape(-1, 28, 28, 1).astype("float32") / 255

In [52]:
y_train[0:3]

22460    8
20828    5
32032    6
Name: label, dtype: int64

In [53]:
y_train = to_categorical(y_train, num_classes=10)
y_val = to_categorical(y_val, num_classes=10)

In [54]:
y_train[0:3]

array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

In [68]:
model = Sequential()
model.add(Conv2D(24,kernel_size=5,padding='same',activation='relu',
        input_shape=(28,28,1)))
model.add(MaxPooling2D())

# model.add(Conv2D(48,kernel_size=5,padding='same',activation='relu'))
# model.add(MaxPooling2D())

model.add(Conv2D(64,kernel_size=5,padding='same',activation='relu'))
model.add(MaxPooling2D(padding='same'))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [69]:
model

In [70]:
h1 = model.fit(X_train,y_train, batch_size=80, epochs = 20, validation_data = (X_val, y_val))

Epoch 1/20
473/473 [==============================] - 18s 36ms/step - loss: 0.2000 - accuracy: 0.9390 - val_loss: 0.0837 - val_accuracy: 0.9740
Epoch 2/20
473/473 [==============================] - 17s 36ms/step - loss: 0.0531 - accuracy: 0.9834 - val_loss: 0.0503 - val_accuracy: 0.9836
Epoch 3/20
473/473 [==============================] - 17s 36ms/step - loss: 0.0364 - accuracy: 0.9886 - val_loss: 0.0340 - val_accuracy: 0.9871
Epoch 4/20
473/473 [==============================] - 17s 36ms/step - loss: 0.0260 - accuracy: 0.9919 - val_loss: 0.0442 - val_accuracy: 0.9857
Epoch 5/20
473/473 [==============================] - 18s 37ms/step - loss: 0.0199 - accuracy: 0.9937 - val_loss: 0.0385 - val_accuracy: 0.9874
Epoch 6/20
473/473 [==============================] - 17s 37ms/step - loss: 0.0152 - accuracy: 0.9952 - val_loss: 0.0382 - val_accuracy: 0.9876
Epoch 7/20
473/473 [==============================] - 18s 38ms/step - loss: 0.0135 - accuracy: 0.9956 - val_loss: 0.0308 - val_accuracy:

In [71]:
results = model.predict(X_test)

875/875 [==============================] - 3s 4ms/step


In [72]:
results = np.argmax(results, axis = 1)
results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results], axis = 1)
submission.to_csv("MNIST-CNN.csv",index=False)